In [1]:
TEST_MODE = 0 # Testing macro

In [13]:
# !pip install --upgrade pip

import os
from sklearn.neighbors import NearestNeighbors, LocalOutlierFactor
import numpy as np # linear algebraf
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.svm import OneClassSVM
from sklearn.model_selection import cross_validate, RandomizedSearchCV, GridSearchCV
from sklearn import svm
from tqdm import tqdm
import warnings
from sklearn.metrics import auc, confusion_matrix, roc_curve
from sklearn.model_selection import KFold
from sklearn.ensemble import IsolationForest


pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
import seaborn as sns

from pandas.testing import assert_frame_equal

%run ./NN_utility-functions.ipynb
%run ./WACA_utility-functions.ipynb
%run ./SEED-CONSTANTS.ipynb


print(f"\x1b[32mSEED: {SEED}\x1b[0m")
# Global utitlity functions are in separate notebook
if TEST_MODE: print("setup complete")

SEED: 567
SEED and CONSTANTS imported
SEED: 567
setup complete
Python 3.9.10
Utility functions imported
SEED: 567
SEED and CONSTANTS imported
SEED: 567
utility_functions imports setup complete
Python 3.9.10
Preprocessing utility functions imported
SEED: 567
SEED and CONSTANTS imported
SEED: 567
utility_functions imports setup complete
Python 3.9.10
Neural Networks utility functions imported
SEED: 567
SEED and CONSTANTS imported
SEED: 567
setup complete
Python 3.9.10
Utility functions imported
SEED: 567
SEED and CONSTANTS imported
SEED: 567
utility_functions imports setup complete
Python 3.9.10
Preprocessing utility functions imported
SEED: 567
SEED and CONSTANTS imported
SEED: 567
utility_functions imports setup complete
Python 3.9.10
WACA utility functions imported
SEED: 567
SEED and CONSTANTS imported
SEED: 567
utility_functions imports setup complete


In [5]:
!python --version

Python 3.9.10
"red"
"green"


In [13]:
def combine_X_exp_dicts(deep_X_exp_dic, WACA_X_exp_dic):
    
    union_dic = {}
    for user in deep_X_exp_dic:
        union_dic[user] = {"profile_windows": np.concatenate([WACA_X_exp_dic[user]["profile_windows"], deep_X_exp_dic[user]["profile_windows"]], axis=1),
                           'unknown_users_dict':{}}
        
        for unknown_user in deep_X_exp_dic[user]['unknown_users_dict']:
            union_dic[user]['unknown_users_dict'][unknown_user] = np.concatenate([WACA_X_exp_dic[user]["unknown_users_dict"][unknown_user], 
                                                                               deep_X_exp_dic[user]["unknown_users_dict"][unknown_user]], axis=1)
            
    return union_dic

In [ ]:
# not used in all files, check where and how its used
def plot_EER_distro(EER_distro_df, discription, save_file_name=None):
    y_col = "EER"
    figsize=(30, 6)
    fontsize=11
    n_subject = len(EER_distro_df.columns)
    mean_col = EER_distro_df[y_col].mean()

    fig = plt.figure(figsize=figsize, dpi=180)
    ax = sns.boxplot(x="owner", y=y_col, data=EER_distro_df)#, **utils_boxplot_style)
    ax.set_ylim((0, 1))
    sns.swarmplot(x="owner", y=y_col, data=EER_distro_df, color=".25")

    plt.plot(
        [-0.6, figsize[0] + 0.6],
        [mean_col, mean_col],
        linestyle="dashed",
        linewidth=1,
        color=MAGENTA,
        alpha=0.7,
    )
    plt.text(n_subject + 0.6, mean_col, f"mean", fontsize=fontsize, color=MAGENTA)
    plt.text(
        n_subject + 0.6, mean_col - 0.04, f"{mean_col:.3f}", fontsize=fontsize, color=MAGENTA
    )
    plt.xticks(rotation=45)
    fig.tight_layout()
    plt.title(discription)
    
    plt.savefig(f'{save_file_name}.png', bbox_inches='tight')
    print(f"Overall mean: {mean_col:.4f}")

# NearestNeighbors classfier, hyperparam optimization and testing

In [ ]:
def make_EER_distro_df(X_exp_dic, n_neighbors, exp_config, algorithm='brute', verbose=0):
    columns = ['owner', "adv_user_id", "EER"]
    EER_distro_df = pd.DataFrame(columns = columns)
    for owner in X_exp_dic.keys():
        
        nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm=algorithm, p=exp_config.p).fit(X_exp_dic[owner]["profile_windows"])
        
        # n_neighbors=n_neighbors+1 since for the owner the first neighbor is the point itself and we drop that
        dist_profile_windows, profile_windows_dist_indices = nbrs.kneighbors(X_exp_dic[owner]["profile_windows"], n_neighbors=n_neighbors+1)
        dist_profile_windows, profile_windows_dist_indices = dist_profile_windows[:, 1:], profile_windows_dist_indices[:, 1:]
        dist_profile_windows = dist_profile_windows.mean(axis=1).reshape(dist_profile_windows.shape[0], 1)
        ## scaler = MinMaxScaler(clip=True).fit(dist_profile_windows)
    

        distFRR, FRR_indices = nbrs.kneighbors(X_exp_dic[owner]["unknown_users_dict"][owner], n_neighbors=n_neighbors)
        distFRR = distFRR.mean(axis=1).reshape(distFRR.shape[0], 1)
        for adv_user_id in X_exp_dic.keys():

            if adv_user_id != owner:
                # print(f"adv_user_id: {adv_user_id}, owner: {owner}")
                distFAR, FAR_indices = nbrs.kneighbors(X_exp_dic[owner]["unknown_users_dict"][adv_user_id], n_neighbors=n_neighbors)
                # print(distFAR)
                
                
                # should axis be 1?
                # can i do this with profile data?
                distFAR = distFAR.mean(axis=1).reshape(distFAR.shape[0], 1)
                # print(distFAR)

                
                ## scaled_distFRR, scaled_distFAR = scaler.transform(distFRR), scaler.transform(distFAR)


                y_true = [1]*distFRR.shape[0] + [-1]*distFAR.shape[0]
                ## y_pred = 1-np.concatenate([scaled_distFRR, scaled_distFAR]).ravel() 
                
                y_score = -1*np.concatenate([distFRR, distFAR]).ravel() 
                
                equal_error_rate(y_true, y_score)
                temp_eer, tres = equal_error_rate(y_true, y_score, return_threshold=True)
                ## temp_eer, tres = utils_eer(y_true, y_pred, return_threshold=True)

                if verbose: print(f"EER: {temp_eer:.9f}, owner_id: {owner}, adv_user_id: {adv_user_id}, Threshold: {tres:.9f}")

                new_row = pd.DataFrame([[owner, adv_user_id, temp_eer]], columns=columns)
                EER_distro_df = pd.concat([EER_distro_df, new_row])
                
                
    
    return EER_distro_df



def calculate_EER_different_window_sizes_optimize_num_neighbors(dfList_dict, window_size_lst, train_set, exp_config, extract_features_func_dict, overlap, n_neighbors_params):
    
    dfList_exp1=dfList_dict["dfList_exp1"]
    dfList_exp2=dfList_dict["dfList_exp2"]
    dfList_exp1_user_47=dfList_dict["dfList_exp1_user_47"]
    dfList_exp2_user_47=dfList_dict["dfList_exp2_user_47"]
    EER_distro_df_dict = {}
    columns = ['window_size', "step_width", "Mean_EER", "best_n_neighbors"]
    Mean_EER_df = pd.DataFrame(columns = columns)
    for window_size in tqdm(window_size_lst):
        print(f"window_size: {window_size}")

        step_width = int(window_size * (1-overlap))

#         X_train_exp1_dict, X_train_exp2_dict, fitted_scaler_train_exp2_dict=get_raw_windows(dfList_exp1, dfList_exp2, window_size, step_width, train_set, exp_config.scaler, 
#                                                                                             exp_config.num_sample_points_per_exp, 
#                                                                                             EMA_per_win_span=exp_config.EMA_per_win_span, 
#                                                                                             SMA_per_win_winsize=exp_config.SMA_per_win_winsize,
#                                                                                             Butter_per_win_argdict=exp_config.Butter_per_win_argdict, 
#                                                                                             verbose=0)

#         X_train_exp1_dict_user_47, X_train_exp2_dict_user_47, fitted_scaler_train_exp2_dict_user_47=get_raw_windows_user_47(dfList_exp1_user_47, dfList_exp2_user_47, 
#                                                                                                                             window_size, step_width, scaler=exp_config.scaler, 
#                                                                                                                             num_sample_points_per_exp=exp_config.num_sample_points_per_exp, 
#                                                                                                                             EMA_per_win_span=exp_config.EMA_per_win_span, 
#                                                                                                                             SMA_per_win_winsize=exp_config.SMA_per_win_winsize,
#                                                                                                                             Butter_per_win_argdict=exp_config.Butter_per_win_argdict, 
#                                                                                                                             verbose=0)

#         X_train_exp1_dict, X_train_exp2_dict, fitted_scaler_train_exp2_dict=append_user_47_to_data(X_train_exp1_dict, X_train_exp2_dict, fitted_scaler_train_exp2_dict, exp_config.user_ids, 
#                                                                                                    X_train_exp1_dict_user_47, X_train_exp2_dict_user_47, fitted_scaler_train_exp2_dict_user_47, 
#                                                                                                    verbose=0)
        
        extract_features_func=extract_features_func_dict[window_size]
        
        X_exp_train_dic = calculate_X_exp_dict(dfList_exp1=dfList_exp1,
                                               dfList_exp2=dfList_exp2,
                                               dfList_exp1_user_47=dfList_exp1_user_47, 
                                               dfList_exp2_user_47=dfList_exp2_user_47,
                                               window_size=window_size,
                                               step_width=step_width,
                                               user_idx_set=train_set,
                                               exp_config=exp_config,
                                               extract_features_func=extract_features_func,
                                               verbose=0)
        
        # X_exp_train_dic = extract_features_func(X_train_exp1_dict, X_train_exp2_dict, fitted_scaler_train_exp2_dict, scaler_clip=exp_config.scaler_clip)


        mean_eer_dict = {}
        for n_neighbors in n_neighbors_params:
            EER_distro_df = make_EER_distro_df(X_exp_train_dic, n_neighbors=n_neighbors, algorithm='brute', exp_config=exp_config, verbose=0)
            EER_distro_df_dict[f"window_size: {window_size}, step_width: {step_width}"] = EER_distro_df

            y_col = "EER"
            mean_col = EER_distro_df[y_col].mean()
            # print(EER_distro_df)
            mean_eer_dict[n_neighbors] = mean_col
            
        # print(mean_eer_dict)
        best_n_neighbors = min(mean_eer_dict, key=mean_eer_dict.get)
        Mean_EER = mean_eer_dict[best_n_neighbors]

        new_row = pd.DataFrame([[window_size, step_width, Mean_EER, best_n_neighbors]], columns=columns)
        Mean_EER_df = pd.concat([Mean_EER_df, new_row])
        # enablePrint()
    return Mean_EER_df


def calculate_EER_different_window_sizes_test(dfList_dict, window_size_lst, test_set, exp_config, extract_features_func_dict, overlap, best_param_df):
    dfList_exp1=dfList_dict["dfList_exp1"]
    dfList_exp2=dfList_dict["dfList_exp2"]
    EER_distro_df_dict = {}
    columns = ['window_size', "step_width", "Mean_EER", "best_n_neighbors"]
    # adding Gini coef column
    columns+=["Gini_coef"]
    Mean_EER_df = pd.DataFrame(columns = columns)
    for window_size in tqdm(window_size_lst):
        print(f"window_size: {window_size}")
        step_width = int(window_size * (1-overlap))
        
        
        # X_test_exp1_dict, X_test_exp2_dict, fitted_scaler_test_exp2_dict=get_raw_windows(dfList_exp1, dfList_exp2, window_size, step_width, test_set, exp_config.scaler, 
        #                                                                                  exp_config.num_sample_points_per_exp, EMA_per_win_span=exp_config.EMA_per_win_span, 
        #                                                                                  SMA_per_win_winsize=exp_config.SMA_per_win_winsize,
        #                                                                                  Butter_per_win_argdict=exp_config.Butter_per_win_argdict, 
        #                                                                                  verbose=0)

        extract_features_func=extract_features_func_dict[window_size]
        
        X_exp_test_dic = calculate_X_exp_dict(dfList_exp1=dfList_exp1,
                                              dfList_exp2=dfList_exp2,
                                              dfList_exp1_user_47=dfList_exp1_user_47, 
                                              dfList_exp2_user_47=dfList_exp2_user_47,
                                              window_size=window_size,
                                              step_width=step_width,
                                              user_idx_set=test_set,
                                              exp_config=exp_config,
                                              extract_features_func=extract_features_func,
                                              verbose=0)
        
        # X_exp_test_dic = extract_features_func(X_test_exp1_dict, X_test_exp2_dict, fitted_scaler_test_exp2_dict, scaler_clip=exp_config.scaler_clip)
        

        
        best_n_neighbors = int(best_param_df[best_param_df.window_size==window_size].best_n_neighbors) # maybe replace with items() 
        EER_distro_df = make_EER_distro_df(X_exp_test_dic, n_neighbors=best_n_neighbors, exp_config=exp_config)
        EER_distro_df_dict[f"window_size: {window_size}, step_width: {step_width}"] = EER_distro_df

        y_col = "EER"
        mean_col = EER_distro_df[y_col].mean()
        Mean_EER = mean_col
        
        gini_coef=gini(EER_distro_df[y_col].to_numpy())

        new_row = pd.DataFrame([[window_size, step_width, Mean_EER, best_n_neighbors, gini_coef]], columns=columns)
        Mean_EER_df = pd.concat([Mean_EER_df, new_row])
        # enablePrint()
        
    return Mean_EER_df#, EER_distro_df_dict

# OCSVM classfier, hyperparam optimization and testing

In [ ]:
def eer_threshold(y_true, y_score):
    fpr, tpr, thresholds = roc_curve(y_true, y_score, pos_label=1)
    fnr = 1 - tpr
    # Find the point where FPR is closest to FNR
    eer_index = np.argmin(np.abs(fpr - fnr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    return thresholds[eer_index]

def utils_eer_scorer5(estimator, X, y_true):
    y_score = estimator.decision_function(X)
    eer = equal_error_rate(y_true, y_score)
    return eer

def accuracy_scorer2(estimator, X, y_true):
    # Calculate the EER threshold using the decision function scores
    y_score = estimator.decision_function(X)
    threshold = eer_threshold(y_true, y_score)

    # Make predictions using the EER threshold
    y_pred = np.where(y_score >= threshold, 1, -1)

    # Calculate the confusion matrix and accuracy
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    return accuracy

def precision_scorer(estimator, X, y_true):
    # Calculate the EER threshold using the decision function scores
    y_score = estimator.decision_function(X)
    threshold = eer_threshold(y_true, y_score)

    # Make predictions using the EER threshold
    y_pred = np.where(y_score >= threshold, 1, -1)

    # Calculate the confusion matrix and precision
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    precision = tp / (tp + fp)
    return precision

def recall_scorer(estimator, X, y_true):
    # Calculate the EER threshold using the decision function scores
    y_score = estimator.decision_function(X)
    threshold = eer_threshold(y_true, y_score)

    # Make predictions using the EER threshold
    y_pred = np.where(y_score >= threshold, 1, -1)

    # Calculate the confusion matrix and recall
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    recall = tp / (tp + fn)
    return recall

def f1_score_scorer(estimator, X, y_true):
    # Calculate the EER threshold using the decision function scores
    y_score = estimator.decision_function(X)
    threshold = eer_threshold(y_true, y_score)

    # Make predictions using the EER threshold
    y_pred = np.where(y_score >= threshold, 1, -1)

    # Calculate the confusion matrix and F1-score
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score



def accuracy_scorer(estimator, X, y):
    # Get the predicted labels
    y_pred = estimator.predict(X)

    # Calculate the confusion matrix
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()

    # Calculate accuracy
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    return accuracy


def equal_error_rate(y_true, y_score, return_threshold=False):
    fpr, tpr, thresholds = roc_curve(y_true, y_score, pos_label=1)
    fnr = 1 - tpr
    # Find the point where FPR is closest to FNR
    eer_index = np.argmin(np.abs(fpr - fnr))
    
    
#     #-----
#     print(f"len(thresholds): {len(thresholds)}")
#     print(fpr[eer_index]-fnr[eer_index])

#     # Plot ROC curve
#     plt.figure()
#     lw = 2
#     plt.plot(thresholds, fpr, color='blue', lw=lw, label='FAR')
#     plt.plot(thresholds, fnr, color='darkorange', lw=lw, label='FRR')
#     plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
#     plt.xlim([min(thresholds), max(thresholds)])
#     plt.ylim([0.0, 1.05])
#     plt.xlabel('threshold')
#     plt.ylabel('error rate')
#     plt.title('Err vs Thres')
#     plt.legend(loc="lower right")
#     plt.show()
#     # -----
    
    eer = (fpr[eer_index] + fnr[eer_index]) / 2
    
    if return_threshold:
        return eer, thresholds[eer_index]

    return eer

# def utils_eer_scorer4(estimator, X, y):
#     y_pred = estimator.decision_function(X)
#     return equal_error_rate(y, y_pred)

utils_eer_scorer4 = make_scorer(equal_error_rate, greater_is_better=False, needs_threshold=True) # OMG needs_threshold=True




def utils_eer3(y_true, y_pred, return_threshold=False):
    """Calculate the Equal Error Rate.

    Parameters:
    y_true (array-like): The true binary labels (1 for normal and -1 for anomalous).
    y_pred (array-like): The predicted anomaly scores from OneClassSVM.
    return_threshold (bool, optional): If True, the threshold value where EER is achieved will be returned.

    Returns:
    float: The equal error rate value.
    float, optional: The threshold value where EER is achieved, if `return_threshold` is set to True.
    """
    # Calculate the ROC curve
    fpr, tpr, thresholds = roc_curve(y_true, y_pred, pos_label=1)

    # Calculate the False Negative Rate (FNR)
    fnr = 1 - tpr

    # Find the point where FPR is closest to FNR
    eer_index = np.argmin(np.abs(fpr - fnr))
    eer = (fpr[eer_index] + fnr[eer_index]) / 2

    if return_threshold:
        threshold = thresholds[eer_index]
        return eer, threshold

    return eer
utils_eer_scorer3 = make_scorer(utils_eer3, greater_is_better=False, needs_threshold=True)




# source: https://github.com/dynobo/ContinAuth/blob/master/notebooks/utils.ipynb
def utils_eer(y_true, y_pred, return_threshold=False):
    """Calculate the Equal Error Rate.

    Based on https://stackoverflow.com/a/49555212, https://yangcha.github.io/EER-ROC/
    and https://scikit-learn.org/stable/modules/model_evaluation.html#implementing-your-own-scoring-object

    Arguments:
        y_true {np.array}  -- Actual labels
        y_pred {np.array}  -- Predicted labels or probability
        
    Returns:
        float              -- Equal Error Rate        
    """
    fpr, tpr, thresholds = roc_curve(y_true, y_pred, pos_label=1)
    eer = brentq(lambda x: 1.0 - x - interp1d(fpr, tpr)(x), 0.0, 1.0)
    thresh = interp1d(fpr, thresholds)(eer)  # Calculated threshold, not needed for score
    if return_threshold:
        return eer, thresh
    else:
        return eer
utils_eer_scorer = make_scorer(utils_eer, greater_is_better=False)#, needs_threshold=True)


In [ ]:
def evaluate_owner_classifier_train_valid(owner_key, X_exp_train_dic, seed, run, param_dist, cores, exp_config, verbose=0):
    run_seed = seed + run
    train_dic, valid_test_dic = { owner_key: X_exp_train_dic[owner_key]["profile_windows"]}, X_exp_train_dic[owner_key]["unknown_users_dict"] 

    X_vals_owner_idx = utils_create_cv_splits(owner_key, train_dic, valid_test_dic, seed=run_seed, verbose=verbose-1) #check

    # X_test_regular = X_vals_owner_idx['X_test_regular'].copy()
    # X_test_anomalous = X_vals_owner_idx['X_test_anomalous'].copy()

    cv_splits = X_vals_owner_idx['cv_splits']

    if exp_config.classifier=="OCSVM":
        clf = OneClassSVM(kernel=exp_config.ocsvm_kernel, 
                          verbose=0)
        
    elif exp_config.classifier=="IF":
        clf = IsolationForest(random_state=run_seed, 
                              n_jobs=cores, 
                              verbose=max(0, verbose-1))
        
    elif exp_config.classifier=="LOF":
        # Create the LOF estimator with novelty detection enabled
        clf = LocalOutlierFactor(novelty=True, 
                                 algorithm="brute", 
                                 metric=exp_config.LOF_metric, 
                                 p=exp_config.p,
                                 n_jobs=cores)
        
    else:
        raise Exception("Unspecified classifier!")
        

    pipeline = Pipeline([
#                             ('scaler', StandardScaler()), 
                        # ('scaler', get_new_scaler_dict[P.scaler]()), 
#                             ('scaler', Normalizer()),
#                              ('pca', pca), 
#                              ('selector', VarianceThreshold()), 
                         ('model', clf)
                        ])

    search = RandomizedSearchCV(
        pipeline,
        param_distributions=param_dist,
        cv=cv_splits,
#             n_iter=120,
        n_iter=80,
#             n_iter=480,
        n_jobs=cores,
        refit=False,
        scoring={"eer": utils_eer_scorer4, "accuracy": accuracy_scorer2},
        verbose=verbose,
        return_train_score=False,
        # iid=False, #invalid arg in sklearn 1.0.2
        error_score="raise",
        random_state=run_seed
    )
#         search = GridSearchCV(
#             pipeline,
#             param_grid=param_dist,
#             cv=cv_splits,
#             n_jobs=cores,
#             refit=False,
#             scoring={"eer": utils_eer_scorer, "accuracy": "accuracy"},
#             verbose=1,
#             return_train_score=False,
#             iid=False,
#             error_score="raise",
#         )
#         print(X_train.shape)
#         print(X_test_anomalous.shape)


    search.fit(X_vals_owner_idx['X_train'], X_vals_owner_idx['y_train'])
    impostors = [user_key for user_key in X_exp_train_dic.keys() if user_key != owner_key]

    df_report = utils_cv_report(search, owner_key, impostors) # check
    df_report["run"] = run
    
    return df_report #{"df_report": df_report, "owner_key": owner_key}


# non parallel is fast enough so i did not use this one. Needs double checking for correcteness before using
def evaluate_owner_classifier_train_test(owner_key, X_exp_test_dic, seed, run, cores, exp_config, verbose=0):
    run_seed = seed + run
    train_dic, valid_test_dic = { owner_key: X_exp_test_dic[owner_key]["profile_windows"]}, X_exp_test_dic[owner_key]["unknown_users_dict"] 

    X_vals_owner_idx = utils_create_cv_splits(owner_key, train_dic, valid_test_dic, seed=run_seed, verbose=verbose-1)

    train_test_cv_splits = X_vals_owner_idx['cv_splits']

    
    # X_test_regular = X_vals_owner_idx['X_test_regular'].copy()
    # X_test_anomalous = X_vals_owner_idx['X_test_anomalous'].copy()

#         print(X_vals_owner_idx['cv_splits'])
#         pca = PCA(n_components = run+3)
#         X_train = pca.fit_transform(X_train)
#         X_test_regular = pca.transform(X_test_regular)
#         X_test_anomalous = pca.transform(X_test_anomalous)

#         pca_fs.add_user_pca(owner_idx, pca)

    if exp_config.classifier=="OCSVM":
        clf = svm.OneClassSVM(kernel=exp_config.ocsvm_kernel, nu=exp_config.ocsvm_nu, gamma=exp_config.ocsvm_gamma, verbose=0)
        
    elif exp_config.classifier=="IF":
        clf = IsolationForest(n_estimators = exp_config.median_n_estimators, 
                              max_samples = exp_config.median_max_samples, 
                              contamination = exp_config.median_contamination, 
                              max_features = exp_config.median_max_features, 
                              random_state=run_seed, n_jobs=cores, verbose=max(0, verbose-1))
    
    elif exp_config.classifier=="LOF":
        # Create the LOF estimator with novelty detection enabled
        clf = LocalOutlierFactor(novelty=True, 
                                 algorithm="brute", 
                                 metric=exp_config.LOF_metric, 
                                 p=exp_config.p,
                                 n_neighbors=exp_config.median_n_neighbors, 
                                 contamination = exp_config.median_contamination, 
                                 n_jobs=cores)
        
    else:
        raise Exception("Unspecified classifier!")
        
    
    pipeline = Pipeline([
                        # ('scaler', get_new_scaler_dict[P.scaler]()), 
#                             ('scaler2', Normalizer()),#best result
#                              ('pca', pca), 
#                              ('selector', VarianceThreshold()), 
                        ('model', clf)
                        ])

    
    scores = cross_validate(
        pipeline,
        X_vals_owner_idx['X_train'],
        X_vals_owner_idx['y_train'],
        cv=train_test_cv_splits,
        scoring={
            "eer": utils_eer_scorer4,
            "accuracy": accuracy_scorer2,
            "precision": precision_scorer,
            "recall": recall_scorer,
            "f1_score": f1_score_scorer,
        },
        n_jobs=cores,
        verbose=verbose,
        return_train_score=False, # train eer is meaningless
    )

    impostors = [user_key for user_key in X_exp_test_dic.keys() if user_key != owner_key]
    df_score = pd.DataFrame(scores)
    df_score["owner"] = owner_key
    # df_score["train_eer"] = df_score["train_eer"].abs()  # train eer is meaningless
    df_score["test_eer"] = df_score["test_eer"].abs()
    df_score["run"] = run
    df_score["impostors"] = impostors
    
    return df_score

def parallel_find_best_hyperparams(X_exp_train_dic, param_dist, exp_config, cores, verbose=0):
    
    df_results = None  # Will be filled with randomsearch scores

    for run in tqdm(range(3)):

        with parallel_backend('loky', n_jobs=-1):
            df_reports = Parallel(n_jobs=-1, verbose=verbose)(delayed(evaluate_owner_classifier_train_valid)(owner_key, X_exp_train_dic, SEED, run, 
                                                                                                         param_dist, cores=cores, exp_config=exp_config) 
                                                          for owner_key in tqdm(
                                                              X_exp_train_dic.keys(),
                                                              desc=f"run: {run}",
                                                          ))
        df_results = pd.concat([df_results] + df_reports, sort=False, ignore_index=True)


    if exp_config.classifier=="OCSVM":
        result_dict = utils_plot_randomsearch_results(df_results, n_top=1, plot=False)
        
    elif exp_config.classifier=="IF":
        result_dict = utils_plot_randomsearch_results_IF(df_results, n_top=1, plot=False)
        
    elif exp_config.classifier=="LOF":
        result_dict = utils_plot_randomsearch_results_LOF(df_results, n_top=1, plot=False)
        
    else:
        raise Exception("Unspecified classifier")
        
    return result_dict
        
def parallel_evaluation_classifier_train_test(X_exp_dic, exp_config, cores, verbose=0):
    """
    return mean_test_EER_df
    """
    test_df_results = None  # Will be filled with randomsearch scores
    for run in tqdm(range(5)):

        with parallel_backend('loky', n_jobs=-1):
            df_score = Parallel(n_jobs=-1, verbose=verbose)(delayed(evaluate_owner_classifier_train_test)(owner_key, X_exp_dic, SEED, run, 
                                                                                                      cores=cores, exp_config=exp_config, verbose=0) 
                                                        for owner_key in tqdm(
                                                            X_exp_dic.keys(),
                                                            desc=f"run: {run}",
                                                        ))
        test_df_results = pd.concat([test_df_results] + df_score, sort=False, ignore_index=True, axis=0)


    mean_test_EER=test_df_results.groupby(['owner', 'impostors'], sort=False)[['fit_time', 'score_time', 'test_eer', 'test_accuracy', 
                                                                               'test_precision', 'test_recall', "test_f1_score"]].mean().reset_index()


    return mean_test_EER

def calculate_X_exp_dict(dfList_exp1,
                         dfList_exp2,
                         dfList_exp1_user_47, 
                         dfList_exp2_user_47,
                         window_size,
                         step_width,
                         user_idx_set,
                         exp_config,
                         extract_features_func,
                         # min_max_scaler=None,
                         verbose=0):
    
    # get user set of all users in the current phase
    user_idx_set=set(user_idx_set)
    
    # separate user 47 which has 29 index by default from the other users as 47 has shorter time series
    if 29 in user_idx_set:
        
        user_idx_set_without_user_47 = user_idx_set - {29}
        
    else:
        
        user_idx_set_without_user_47 = user_idx_set
    
    # if this is for SCNN exp, this version has a min_max_scaler unlike WACA
    if exp_config.is_NN:
        print("NN preprocessing")
        
        X_exp1_dict, X_exp2_dict, \
        fitted_scaler_exp2_dict, fitted_min_max_scaler_exp2_dict=get_raw_windows_NN(\
                                                                                    dfList_exp1=dfList_exp1, 
                                                                                    dfList_exp2=dfList_exp2, 
                                                                                    window_size=window_size, 
                                                                                    step_width=step_width, 
                                                                                    user_idx_set=user_idx_set_without_user_47, 
                                                                                    scaler=exp_config.scaler, 
                                                                                    min_max_scaler=True,
                                                                                    num_sample_points_per_exp=exp_config.num_sample_points_per_exp, 
                                                                                    EMA_per_win_span=exp_config.EMA_per_win_span, 
                                                                                    SMA_per_win_winsize=exp_config.SMA_per_win_winsize,
                                                                                    Butter_per_win_argdict=exp_config.Butter_per_win_argdict, 
                                                                                    verbose=verbose)

        # if user 47 is present in the current phase, extract its data and then merge it with other users data
        if 29 in user_idx_set:
            
            X_exp1_dict_user_47, X_exp2_dict_user_47, \
            fitted_scaler_exp2_dict_user_47, \
            fitted_min_max_scaler_exp2_dict_user_47=get_raw_windows_user_47_NN(\
                                                                               dfList_exp1_user_47=dfList_exp1_user_47, 
                                                                               dfList_exp2_user_47=dfList_exp2_user_47, 
                                                                               window_size=window_size, 
                                                                               step_width=step_width, 
                                                                               scaler=exp_config.scaler, 
                                                                               min_max_scaler=True,
                                                                               num_sample_points_per_exp=exp_config.num_sample_points_per_exp,
                                                                               EMA_per_win_span=exp_config.EMA_per_win_span, 
                                                                               SMA_per_win_winsize=exp_config.SMA_per_win_winsize,
                                                                               Butter_per_win_argdict=exp_config.Butter_per_win_argdict, 
                                                                               verbose=verbose)

            X_exp1_dict, X_exp2_dict, \
            fitted_scaler_exp2_dict, \
            fitted_min_max_scaler_exp2_dict=append_user_47_to_data_NN(\
                                                                      X_exp1_dict=X_exp1_dict, 
                                                                      X_exp2_dict=X_exp2_dict,
                                                                      fitted_scaler_exp2_dict=fitted_scaler_exp2_dict, 
                                                                      all_user_set=user_idx_set_without_user_47,
                                                                      X_exp1_dict_user_47=X_exp1_dict_user_47, 
                                                                      X_exp2_dict_user_47=X_exp2_dict_user_47, 
                                                                      fitted_scaler_exp2_dict_user_47=fitted_scaler_exp2_dict_user_47, 
                                                                      fitted_min_max_scaler_exp2_dict=fitted_min_max_scaler_exp2_dict, 
                                                                      fitted_min_max_scaler_exp2_dict_user_47=fitted_min_max_scaler_exp2_dict_user_47,
                                                                      verbose=verbose)
        


        X_features_dict = extract_features_func(X_exp1_dic=X_exp1_dict, 
                                                X_exp2_dic=X_exp2_dict, 
                                                fitted_scaler_classifier_exp2_dic=fitted_scaler_exp2_dict, 
                                                fitted_min_max_scaler_exp2_dict=fitted_min_max_scaler_exp2_dict,
                                                scaler_clip=exp_config.scaler_clip)
        
    # if this is for a WACA exp
    else:
        
        print("WACA preprocessing")
        
        X_exp1_dict, X_exp2_dict, fitted_scaler_exp2_dict=get_raw_windows(\
                                                                          dfList_exp1=dfList_exp1, 
                                                                          dfList_exp2=dfList_exp2, 
                                                                          window_size=window_size, 
                                                                          step_width=step_width, 
                                                                          user_idx_set=user_idx_set_without_user_47, 
                                                                          scaler=exp_config.scaler, 
                                                                          num_sample_points_per_exp=exp_config.num_sample_points_per_exp, 
                                                                          EMA_per_win_span=exp_config.EMA_per_win_span, 
                                                                          SMA_per_win_winsize=exp_config.SMA_per_win_winsize,
                                                                          Butter_per_win_argdict=exp_config.Butter_per_win_argdict, 
                                                                          verbose=verbose)

        # if user 47 is present in the current phase, extract its data and then merge it with other users data
        if 29 in user_idx_set:
            
            X_exp1_dict_user_47, X_exp2_dict_user_47,\
            fitted_scaler_exp2_dict_user_47=get_raw_windows_user_47(dfList_exp1_user_47=dfList_exp1_user_47, 
                                                                    dfList_exp2_user_47=dfList_exp2_user_47, 
                                                                    window_size=window_size, 
                                                                    step_width=step_width, 
                                                                    scaler=exp_config.scaler, 
                                                                    num_sample_points_per_exp=exp_config.num_sample_points_per_exp, 
                                                                    EMA_per_win_span=exp_config.EMA_per_win_span, 
                                                                    SMA_per_win_winsize=exp_config.SMA_per_win_winsize,
                                                                    Butter_per_win_argdict=exp_config.Butter_per_win_argdict, 
                                                                    verbose=verbose)

            X_exp1_dict, X_exp2_dict, fitted_scaler_exp2_dict=append_user_47_to_data(X_exp1_dict=X_exp1_dict, 
                                                                                     X_exp2_dict=X_exp2_dict, 
                                                                                     fitted_scaler_exp2_dict=fitted_scaler_exp2_dict, 
                                                                                     all_user_set=user_idx_set_without_user_47, 
                                                                                     X_exp1_dict_user_47=X_exp1_dict_user_47, 
                                                                                     X_exp2_dict_user_47=X_exp2_dict_user_47, 
                                                                                     fitted_scaler_exp2_dict_user_47=fitted_scaler_exp2_dict_user_47, 
                                                                                     verbose=verbose)

        X_features_dict = extract_features_func(X_exp1_dic=X_exp1_dict, 
                                                X_exp2_dic=X_exp2_dict, 
                                                fitted_scaler_classifier_exp2_dic=fitted_scaler_exp2_dict, 
                                                scaler_clip=exp_config.scaler_clip)


    return X_features_dict
    
    
def calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst, train_set, exp_config, extract_features_func_dict, overlap, param_dist, verbose=0):
    
    dfList_exp1=dfList_dict["dfList_exp1"]
    dfList_exp2=dfList_dict["dfList_exp2"]
    dfList_exp1_user_47=dfList_dict["dfList_exp1_user_47"]
    dfList_exp2_user_47=dfList_dict["dfList_exp2_user_47"]
    EER_distro_df_dict = {}
    
    if exp_config.classifier=="OCSVM":
        columns = ['window_size', "step_width", "Mean_EER", "median_nu", "median_gamma"]

    elif exp_config.classifier=="IF":
        columns = ['window_size', "step_width", "Mean_EER", "median_n_estimators", "median_max_samples", "median_contamination", "median_max_features"]
        
    elif exp_config.classifier=="LOF":
        columns = ['window_size', "step_width", "Mean_EER", "median_n_neighbors", "median_contamination"]

    else:
        raise Exception("Unspecified classifier")
    
    Mean_EER_df = pd.DataFrame(columns = columns)
    for window_size in tqdm(window_size_lst):
        print(f"window_size: {window_size}")

        step_width = int(window_size * (1-overlap))

#         X_train_exp1_dict, X_train_exp2_dict, fitted_scaler_train_exp2_dict=get_raw_windows(dfList_exp1, dfList_exp2, window_size, step_width, train_set, exp_config.scaler, 
#                                                                                             exp_config.num_sample_points_per_exp, 
#                                                                                             EMA_per_win_span=exp_config.EMA_per_win_span, 
#                                                                                             SMA_per_win_winsize=exp_config.SMA_per_win_winsize,
#                                                                                             Butter_per_win_argdict=exp_config.Butter_per_win_argdict, 
#                                                                                             verbose=0)

#         X_train_exp1_dict_user_47, X_train_exp2_dict_user_47, fitted_scaler_train_exp2_dict_user_47=get_raw_windows_user_47(dfList_exp1_user_47, dfList_exp2_user_47, 
#                                                                                                                             window_size, step_width, scaler=exp_config.scaler, 
#                                                                                                                             num_sample_points_per_exp=exp_config.num_sample_points_per_exp, 
#                                                                                                                             EMA_per_win_span=exp_config.EMA_per_win_span, 
#                                                                                                                             SMA_per_win_winsize=exp_config.SMA_per_win_winsize,
#                                                                                                                             Butter_per_win_argdict=exp_config.Butter_per_win_argdict, 
#                                                                                                                             verbose=0)

#         X_train_exp1_dict, X_train_exp2_dict, fitted_scaler_train_exp2_dict=append_user_47_to_data(X_train_exp1_dict, X_train_exp2_dict, fitted_scaler_train_exp2_dict, exp_config.user_ids, 
#                                                                                                    X_train_exp1_dict_user_47, X_train_exp2_dict_user_47, fitted_scaler_train_exp2_dict_user_47, 
#                                                                                                    verbose=0)

# ###added----
#         X_train_exp1_dict, X_train_exp2_dict, \
#         fitted_scaler_train_exp2_dict, fitted_min_max_scaler_train_exp2_dict=get_raw_windows_NN(\
#                                                                                                 dfList_exp1=dfList_exp1, 
#                                                                                                 dfList_exp2=dfList_exp2, 
#                                                                                                 window_size=window_size, 
#                                                                                                 step_width=step_width, 
#                                                                                                 user_idx_set=train_set, 
#                                                                                                 scaler=exp_config.scaler, 
#                                                                                                 min_max_scaler=True,
#                                                                                                 num_sample_points_per_exp=exp_config.num_sample_points_per_exp, 
#                                                                                                 EMA_per_win_span=exp_config.EMA_per_win_span, 
#                                                                                                 SMA_per_win_winsize=exp_config.SMA_per_win_winsize,
#                                                                                                 Butter_per_win_argdict=exp_config.Butter_per_win_argdict, 
#                                                                                                 verbose=0)

#         X_train_exp1_dict_user_47, X_train_exp2_dict_user_47, \
#         fitted_scaler_train_exp2_dict_user_47, \
#         fitted_min_max_scaler_train_exp2_dict_user_47=get_raw_windows_user_47_NN(\
#                                                                                  dfList_exp1_user_47=dfList_exp1_user_47, 
#                                                                                  dfList_exp2_user_47=dfList_exp2_user_47, 
#                                                                                  window_size=window_size, 
#                                                                                  step_width=step_width, 
#                                                                                  scaler=exp_config.scaler, 
#                                                                                  min_max_scaler=True,
#                                                                                  num_sample_points_per_exp=exp_config.num_sample_points_per_exp, 
#                                                                                  EMA_per_win_span=exp_config.EMA_per_win_span, 
#                                                                                  SMA_per_win_winsize=exp_config.SMA_per_win_winsize,
#                                                                                  Butter_per_win_argdict=exp_config.Butter_per_win_argdict, 
#                                                                                  verbose=0)

#         X_train_exp1_dict, X_train_exp2_dict, \
#         fitted_scaler_train_exp2_dict, \
#         fitted_min_max_scaler_train_exp2_dict=append_user_47_to_data_NN(\
#                                                                   X_exp1_dict=X_train_exp1_dict, 
#                                                                   X_exp2_dict=X_train_exp2_dict, 
#                                                                   fitted_scaler_exp2_dict=fitted_scaler_train_exp2_dict, 
#                                                                   all_user_set=exp_config.user_ids, 
#                                                                   X_exp1_dict_user_47=X_train_exp1_dict_user_47, 
#                                                                   X_exp2_dict_user_47=X_train_exp2_dict_user_47, 
#                                                                   fitted_scaler_exp2_dict_user_47=fitted_scaler_train_exp2_dict_user_47, 
#                                                                   fitted_min_max_scaler_exp2_dict=fitted_min_max_scaler_train_exp2_dict, 
#                                                                   fitted_min_max_scaler_exp2_dict_user_47=fitted_min_max_scaler_train_exp2_dict_user_47,
#                                                                   verbose=0)
        
# ###added----


        extract_features_func=extract_features_func_dict[window_size]
    
        X_exp_train_dic = calculate_X_exp_dict(dfList_exp1=dfList_exp1,
                                               dfList_exp2=dfList_exp2,
                                               dfList_exp1_user_47=dfList_exp1_user_47, 
                                               dfList_exp2_user_47=dfList_exp2_user_47,
                                               window_size=window_size,
                                               step_width=step_width,
                                               user_idx_set=train_set,
                                               exp_config=exp_config,
                                               extract_features_func=extract_features_func,
                                               verbose=0)

        # X_exp_train_dic = extract_features_func(X_train_exp1_dict, X_train_exp2_dict, fitted_scaler_classifier_exp2_dic=fitted_scaler_train_exp2_dict, 
        #                                         fitted_min_max_scaler_exp2_dict=fitted_min_max_scaler_train_exp2_dict,#added
        #                                         scaler_clip=exp_config.scaler_clip)


        result_dict = parallel_find_best_hyperparams(X_exp_train_dic, param_dist, exp_config, cores=exp_config.train_cores, verbose=verbose)
        
        if exp_config.classifier=="OCSVM":
            exp_config.ocsvm_nu, exp_config.ocsvm_gamma = result_dict["median_nu"], result_dict["median_gamma"]

        elif exp_config.classifier=="IF":
            exp_config.median_n_estimators, exp_config.median_max_samples, exp_config.median_contamination, exp_config.median_max_features = \
            result_dict["median_n_estimators"], result_dict["median_max_samples"], result_dict["median_contamination"], result_dict["median_max_features"]
            
            exp_config.median_n_estimators = np.int64(np.round(exp_config.median_n_estimators))
        
        elif exp_config.classifier=="LOF":
            exp_config.median_n_neighbors, exp_config.median_contamination = result_dict["median_n_neighbors"], result_dict["median_contamination"]
            
            exp_config.median_n_neighbors = np.int64(np.round(exp_config.median_n_neighbors))
            
        else:
            raise Exception("Unspecified classifier")
        
        
        Mean_valid_EER_df_results = parallel_evaluation_classifier_train_test(X_exp_train_dic, exp_config, cores=exp_config.train_cores, verbose=verbose)
        Mean_EER = Mean_valid_EER_df_results.test_eer.mean()

        
        if exp_config.classifier=="OCSVM":
            new_row = pd.DataFrame([[window_size, step_width, Mean_EER, exp_config.ocsvm_nu, exp_config.ocsvm_gamma]], columns=columns)

        elif exp_config.classifier=="IF":
            new_row = pd.DataFrame([[window_size, step_width, Mean_EER, exp_config.median_n_estimators, exp_config.median_max_samples, 
                                     exp_config.median_contamination, exp_config.median_max_features]], columns=columns)
            
        elif exp_config.classifier=="LOF":
            new_row = pd.DataFrame([[window_size, step_width, Mean_EER, exp_config.median_n_neighbors, exp_config.median_contamination]], columns=columns)
            
        else:
            raise Exception("Unspecified classifier")
        
        Mean_EER_df = pd.concat([Mean_EER_df, new_row])


    return Mean_EER_df


def calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst, test_set, exp_config, extract_features_func_dict, overlap, best_param_df, verbose=0):
    dfList_exp1=dfList_dict["dfList_exp1"]
    dfList_exp2=dfList_dict["dfList_exp2"]
    EER_distro_df_dict = {}

    if exp_config.classifier=="OCSVM":
        columns = ['window_size', "step_width", "Mean_EER", "median_nu", "median_gamma"]

    elif exp_config.classifier=="IF":
        columns = ['window_size', "step_width", "Mean_EER", "median_n_estimators", "median_max_samples", "median_contamination", "median_max_features"]

    elif exp_config.classifier=="LOF":
        columns = ['window_size', "step_width", "Mean_EER", "median_n_neighbors", "median_contamination"]
        
    else:
        raise Exception("Unspecified classifier")
    
    # adding Gini coef column
    columns+=["Gini_coef"]
    
    Mean_EER_df = pd.DataFrame(columns = columns)
    for window_size in tqdm(window_size_lst):
        print(f"window_size: {window_size}")
        step_width = int(window_size * (1-overlap))
        
        
#         X_test_exp1_dict, X_test_exp2_dict, fitted_scaler_test_exp2_dict=get_raw_windows(dfList_exp1, dfList_exp2, window_size, step_width, test_set, exp_config.scaler, 
#                                                                                          exp_config.num_sample_points_per_exp, EMA_per_win_span=exp_config.EMA_per_win_span, 
#                                                                                          SMA_per_win_winsize=exp_config.SMA_per_win_winsize,
#                                                                                          Butter_per_win_argdict=exp_config.Butter_per_win_argdict, 
#                                                                                          verbose=0)

        
#         extract_features_func=extract_features_func_dict[window_size]
        
#         X_exp_test_dic = extract_features_func(X_test_exp1_dict, X_test_exp2_dict, fitted_scaler_test_exp2_dict, scaler_clip=exp_config.scaler_clip)

###---added
        # X_test_exp1_dict, X_test_exp2_dict, \
        # fitted_scaler_test_exp2_dict, fitted_min_max_scaler_test_exp2_dict=get_raw_windows_NN(\
        #                                                                                         dfList_exp1=dfList_exp1, 
        #                                                                                         dfList_exp2=dfList_exp2, 
        #                                                                                         window_size=window_size, 
        #                                                                                         step_width=step_width, 
        #                                                                                         user_idx_set=test_set, 
        #                                                                                         scaler=exp_config.scaler, 
        #                                                                                         min_max_scaler=True,
        #                                                                                         num_sample_points_per_exp=exp_config.num_sample_points_per_exp, 
        #                                                                                         EMA_per_win_span=exp_config.EMA_per_win_span, 
        #                                                                                         SMA_per_win_winsize=exp_config.SMA_per_win_winsize,
        #                                                                                         Butter_per_win_argdict=exp_config.Butter_per_win_argdict, 
        #                                                                                         verbose=0)


        extract_features_func=extract_features_func_dict[window_size]
        
        X_exp_test_dic = calculate_X_exp_dict(dfList_exp1=dfList_exp1,
                                              dfList_exp2=dfList_exp2,
                                              dfList_exp1_user_47=dfList_exp1_user_47, 
                                              dfList_exp2_user_47=dfList_exp2_user_47,
                                              window_size=window_size,
                                              step_width=step_width,
                                              user_idx_set=test_set,
                                              exp_config=exp_config,
                                              extract_features_func=extract_features_func,
                                              verbose=0)

        # X_exp_test_dic = extract_features_func(X_test_exp1_dict, X_test_exp2_dict, fitted_scaler_classifier_exp2_dic=fitted_scaler_test_exp2_dict, 
        #                                         fitted_min_max_scaler_exp2_dict=fitted_min_max_scaler_test_exp2_dict,
        #                                         scaler_clip=exp_config.scaler_clip)
###---added
        
        
        winsize_best_param_df = best_param_df[best_param_df.window_size==window_size]
        if exp_config.classifier=="OCSVM":
            exp_config.ocsvm_nu, exp_config.ocsvm_gamma = winsize_best_param_df.median_nu.item(), winsize_best_param_df.median_gamma.item()

        elif exp_config.classifier=="IF":
            exp_config.median_n_estimators, exp_config.median_max_samples, exp_config.median_contamination, exp_config.median_max_features = \
            winsize_best_param_df.median_n_estimators.item(), winsize_best_param_df.median_max_samples.item(), winsize_best_param_df.median_contamination.item(), \
            winsize_best_param_df.median_max_features.item()
            
        elif exp_config.classifier=="LOF":
            exp_config.median_n_neighbors, exp_config.median_contamination= winsize_best_param_df.median_n_neighbors.item(), winsize_best_param_df.median_contamination.item()
            
        else:
            raise Exception("Unspecified classifier")
            
            
        Mean_test_EER_df_results = parallel_evaluation_classifier_train_test(X_exp_test_dic, exp_config, cores=exp_config.test_cores, verbose=verbose)
        Mean_EER = Mean_test_EER_df_results.test_eer.mean()
        
        gini_coef=gini(Mean_test_EER_df_results.test_eer.to_numpy())


        if exp_config.classifier=="OCSVM":
            new_row = pd.DataFrame([[window_size, step_width, Mean_EER, exp_config.ocsvm_nu, exp_config.ocsvm_gamma, gini_coef]], columns=columns)

        elif exp_config.classifier=="IF":
            new_row = pd.DataFrame([[window_size, step_width, Mean_EER, exp_config.median_n_estimators, exp_config.median_max_samples, 
                                     exp_config.median_contamination, exp_config.median_max_features, gini_coef]], columns=columns)
            
        elif exp_config.classifier=="LOF":
            new_row = pd.DataFrame([[window_size, step_width, Mean_EER, exp_config.median_n_neighbors, exp_config.median_contamination, gini_coef]], columns=columns)
            
        else:
            raise Exception("Unspecified classifier")
        
        Mean_EER_df = pd.concat([Mean_EER_df, new_row])


    return Mean_EER_df

In [51]:
def OneClassSVMSets(k, X_exp1_dic, X_exp2_dic, cv=5):
    '''
    return the required sets for an OCSVM trained on the user with key. 
    X_train: X data from X_exp1_dic[k]
    X_test_regular: X data from X_exp2_dic[k]
    X_test_anomalous: X data from X_exp2_dic[!k]
    '''
    
    if k not in  X_exp1_dic:
        raise Exception("invalid key for dic")
        
    
    X_pos = X_exp1_dic[k]
#     X_neg = np.concatenate([X_exp1_dic[key] for key in X_exp1_dic.keys() if key != k], axis=0)
    X_test_regular = X_exp2_dic[k]
    X_test_anomalous = np.concatenate([X_exp2_dic[key] for key in X_exp2_dic.keys() if key != k], axis=0)
    
    
#     n, m = len(Xpos), len(Xneg)
    np.random.shuffle(X_neg)
    print((X_neg.shape[0], X_pos.shape[0]))
    X_neg = X_neg[np.random.choice(X_neg.shape[0], size=X_pos.shape[0], replace=False), :]
    print(X_pos.shape, X_neg.shape)
    # Creating (train, test) tuples of indices for k-folds cross-validation
    # We split the positive class (normal data) as we only want the positive examples in the training set.
    
    train_splits = KFold(n_splits=cv, shuffle=True).split(X_pos)
    anomalous_splits = KFold(n_splits=cv, shuffle=True).split(X_neg)

#     print(len(train_splits), len(anomalous_splits))
    # Negative examples (abnormal data) are added to the test set (see https://stackoverflow.com/a/58459322/3673842)
    y_train = np.concatenate([np.repeat(1.0, len(X_pos)), np.repeat(-1.0, len(X_neg))])
    X_train = np.concatenate([X_pos, X_neg], axis=0)
    
    # https://github.com/steppi/adeft/blob/anomaly_detection/adeft/modeling/find_anomalies.py#L170
    cv_splits = ((train, np.concatenate((test, anom_test + X_pos.shape[0]), axis = 0))
                  for (train, test), (_, anom_test)
                  in zip(train_splits, anomalous_splits))
    
    return {"X_train": X_train, "y_train": y_train, "X_test_regular": X_test_regular, "X_test_anomalous": X_test_anomalous, "cv_splits": cv_splits}

# utils_create_cv_splits()

In [56]:
def utils_create_cv_splits(owner_key, train_dic, valid_test_dic, seed, verbose=0):
    '''
    return the required sets for an OCSVM trained on the user with key. 
    X_train: X data from train_dic[k], comes from exp2
    X_test_regular: X data from valid_test_dic[k], comes from exp1
    X_test_anomalous: X data from valid_test_dic[!k], comes from exp1
    
    Create cross-validation mask with train-valid pairs.
    
    See e.g. https://stackoverflow.com/a/37591377
    
    Arguments:
        cv_mask {np.ndarray} --
        
    Return:
        {list} -- List of tuple: (<train indices>, <valid indices>)
        
    '''
    
    if owner_key not in  train_dic:
        raise Exception("invalid key for dic")
    
        
    X_pos = train_dic[owner_key].copy()
    X_test_regular = valid_test_dic[owner_key].copy()
    X_test_anomalous = np.concatenate([valid_test_dic[key] for key in valid_test_dic.keys() if key != owner_key], axis=0).copy()
    
    train_idx_owner = np.arange(X_pos.shape[0])
    valid_idx_owner = np.arange(X_test_regular.shape[0]) + train_idx_owner.shape[0]
    
    if verbose>0: print(f"owner: {owner_key} train_idx range: {train_idx_owner[0]}, {train_idx_owner[-1]}")
    if verbose>0: print(f"owner: {owner_key} valid_idx range: {valid_idx_owner[0]}, {valid_idx_owner[-1]}")
    np.random.seed(seed + owner_key)
    np.random.shuffle(train_idx_owner)
    
    np.random.seed(seed + owner_key)
    np.random.shuffle(valid_idx_owner)

    
    cv_splits = []
    base_idx = train_idx_owner.shape[0] + valid_idx_owner.shape[0]
    if verbose>0: print(f"owner key: {owner_key}")
    for key in valid_test_dic.keys():
        
        if key != owner_key:
            # Impostor validation indices
            valid_idx_impostor = np.arange(valid_test_dic[key].shape[0]) + base_idx
            if verbose>0: print(f"imposter: {key} valid_idx range: {valid_idx_impostor[0]}, {valid_idx_impostor[-1]}")

            # Balance classes
            min_samples = min(valid_idx_owner.shape[0], valid_idx_impostor.shape[0])
            np.random.seed(seed + key)
            valid_idx_owner_samp = np.random.choice(
                valid_idx_owner, size=min_samples, replace=False
            )
            np.random.seed(seed + key)
            valid_idx_impostor_samp = np.random.choice(
                valid_idx_impostor, size=min_samples, replace=False
            )

            # Concat owner & impostor validation indices
            valid_idx_both = np.hstack([valid_idx_owner_samp, valid_idx_impostor_samp])

            # Add train/valid pair to cv
            cv_splits.append((list(train_idx_owner), list(valid_idx_both)))
            
            base_idx += valid_idx_impostor.shape[0]


    

    y_train = np.concatenate([np.repeat(1.0, X_pos.shape[0]), np.repeat(1.0, X_test_regular.shape[0]), np.repeat(-1.0, X_test_anomalous.shape[0])])
    X_train = np.concatenate([X_pos, X_test_regular, X_test_anomalous], axis=0)
    
    
    return {"X_train": X_train, "y_train": y_train, "X_test_regular": X_test_regular, "X_test_anomalous": X_test_anomalous, "cv_splits": cv_splits}

In [57]:
if TEST_MODE:
    # Mask Explained:
    # -2 => Training data (owner)
    # -1 => Validation data (owner)
    # 0+ => Validation impostors
    #              Indices:    0   1   2   3   4   5  6  7  8  9  10 11 12 13 14 15
    dummy_cv_mask = np.array([-2, -2, -1, -1, -1, -1, 0, 0, 0, 1, 1, 1, 2, 2, 2, -2])

    # Generate tuples of training data and validation data, one tuple for each impostor (0, 1, 2).
    # Training data (1st list in tuple) contains only indices of owner training data (-2)
    # Validation data (2nd list in tuple) contains  indices of validation data from owner (-1) and
    # from a single impostor (0+), each 50 %
    splits = utils_create_cv_splits(dummy_cv_mask, seed=123)
    [print(s) for s in splits]

# utils_cv_report()

In [58]:
TEST_MODE=0

In [59]:
def utils_cv_report(random_search, owner, impostors, verbose=0):
    """Transform the random_search.cv_results_ into nice formatted dataframe."""
    # Create report
    df_report = pd.DataFrame(random_search.cv_results_)

    # Add owner information
    df_report["owner"] = owner

    # Drop uninteressting columns
    drop_columns = [col for col in df_report.columns if "_train_" in col]
    drop_columns = drop_columns + [col for col in df_report.columns if col.startswith("split") and (col.endswith("recall") or col.endswith("precision") or col.endswith("f1") or col.endswith("roc_auc"))]
    drop_columns = drop_columns + ["params"]
    df_report = df_report.drop(columns=drop_columns)

    # Flip sign of eer (revert flip by sklearn scorer)
    eer_columns = [col for col in df_report.columns if col.endswith("_eer")]
    df_report[eer_columns] = df_report[eer_columns].abs()
    
    # Rename split result columns with impostor-ids used in split
    rename_cols = {}
    for idx, impostor in enumerate(impostors):
        if verbose: print(f"idx: {idx}, impostor: {impostor}")
        to_rename_cols = [col for col in df_report.columns if col.startswith(f"split{idx}")]
        for col in to_rename_cols:
            rename_cols[col] = str(impostor)+col[len(f"split{idx}"):]
    df_report = df_report.rename(columns=rename_cols)      

    return df_report

In [60]:
if TEST_MODE:
    print("Performing Dummy RandomSearch...")
    from sklearn import svm, datasets
    from sklearn.model_selection import RandomizedSearchCV

    iris = datasets.load_iris()
    parameters = {"kernel": ("linear", "rbf"), "C": [1, 2, 3, 4, 5, 6, 7, 10]}
    svc = svm.SVC(gamma="scale")
    clf = RandomizedSearchCV(svc, parameters, cv=3, iid=False)
    clf.fit(iris.data, iris.target)
    print("Create report:")
    df_temp = utils_cv_report(clf, "owner x", ["impo_1", "impo_2", "impo_3"])
    display(df_temp)

In [64]:
def utils_plot_randomsearch_results(df_results, n_top=1, h1_min_max_scale_tuple=(-.1, 1.1, "linear"), h2_min_max_scale_tuple=(1e-9, 1e3, "log"), plot=False):
    # Prepare data for plotting
    sorted_df_results=df_results[(df_results["rank_test_eer"] <= n_top)].sort_values(by=['owner', 'run', 'param_model__nu', "param_model__gamma"], ascending=[True, True, True, True])
    groupted_df_results = sorted_df_results[(sorted_df_results["rank_test_eer"] <= n_top)].groupby(['owner', 'run']).first()
    
    
    df_plot = groupted_df_results[groupted_df_results["rank_test_eer"] <= n_top].rename(
        columns={
            "param_model__nu": r"$\nu$",
            "param_model__gamma": r"$\gamma$",
            "mean_test_accuracy": "Mean Test Acc.",
            "mean_test_eer": "Mean Test EER",
        }
    )
    df_plot["Mean Test EER"] = df_plot["Mean Test EER"] * -1  # Because fewer is more
    
    median_nu = df_plot[r"$\nu$"].median()
    median_gamma = df_plot[r"$\gamma$"].median()
    
    fig=None
    if plot:
        fig = utils_plot_2d_hyperparam_selection(h1_name=r"$\nu$", h1_val=median_nu, h2_name=r"$\gamma$", h2_val=median_gamma, df_plot=df_plot, 
                                                 h1_min_max_scale_tuple=h1_min_max_scale_tuple, h2_min_max_scale_tuple=h2_min_max_scale_tuple)

    return {"median_nu": median_nu, "median_gamma": median_gamma, "fig": fig}


def utils_plot_randomsearch_results_IF(df_results, n_top=1, plot=False):
    
    sorted_df_results=df_results[(df_results["rank_test_eer"] <= n_top)].sort_values(
        by=['owner', 'run', 'param_model__n_estimators', "param_model__max_samples", "param_model__contamination", "param_model__max_features"], 
        ascending=[True, True, True, True, True, True])
    groupted_df_results = sorted_df_results[(sorted_df_results["rank_test_eer"] <= n_top)].groupby(['owner', 'run']).first()
    
    # Prepare data for plotting
    df_plot = df_results[df_results["rank_test_eer"] <= n_top].rename(
        columns={
                "param_model__n_estimators": "n_estimators",
                "param_model__max_samples": "max_samples",
                "param_model__contamination": "contamination",
                "param_model__max_features": "max_features",
                "mean_test_accuracy": "Mean Test Acc.",
                "mean_test_eer": "Mean Test EER",
        }
    )
    df_plot["Mean Test EER"] = df_plot["Mean Test EER"] * -1  # Because fewer is more

    median_n_estimators = df_plot["n_estimators"].median()
    median_max_samples = df_plot["max_samples"].median()
    median_contamination = df_plot["contamination"].median()
    median_max_features = df_plot["max_features"].median()
    
    figures=None
    if plot:
        figures = []
        figures.append(utils_plot_2d_hyperparam_selection("n_estimators", median_n_estimators, "max_samples", median_max_samples, df_plot,
                                                         h1_min_max_scale_tuple=(-10, 500, "linear"), h2_min_max_scale_tuple=(-.1, 1.1, "linear")))
        figures.append(utils_plot_2d_hyperparam_selection("n_estimators", median_n_estimators, "contamination", median_contamination, df_plot,
                                                         h1_min_max_scale_tuple=(-10, 500, "linear"), h2_min_max_scale_tuple=(-.1, 1.1, "linear")))
        figures.append(utils_plot_2d_hyperparam_selection("n_estimators", median_n_estimators, "max_features", median_max_features, df_plot,
                                                         h1_min_max_scale_tuple=(-10, 500, "linear"), h2_min_max_scale_tuple=(-.1, 1.1, "linear")))
        figures.append(utils_plot_2d_hyperparam_selection("max_samples", median_max_samples, "contamination", median_contamination, df_plot,
                                                         h1_min_max_scale_tuple=(-.1, 1.1, "linear"), h2_min_max_scale_tuple=(-.1, 1.1, "linear")))
        figures.append(utils_plot_2d_hyperparam_selection("max_samples", median_max_samples, "max_features", median_max_features, df_plot,
                                                         h1_min_max_scale_tuple=(-.1, 1.1, "linear"), h2_min_max_scale_tuple=(-.1, 1.1, "linear")))
        figures.append(utils_plot_2d_hyperparam_selection("contamination", median_contamination, "max_features", median_max_features, df_plot,
                                                         h1_min_max_scale_tuple=(-.1, 1.1, "linear"), h2_min_max_scale_tuple=(-.1, 1.1, "linear")))
    

    return {"median_n_estimators": median_n_estimators, "median_max_samples": median_max_samples, "median_contamination": median_contamination, 
            "median_max_features": median_max_features, "figures": figures}

def utils_plot_randomsearch_results_LOF(df_results, n_top=1, plot=False):
    
    sorted_df_results=df_results[(df_results["rank_test_eer"] <= n_top)].sort_values(
        by=['owner', 'run', 'param_model__n_neighbors', "param_model__contamination"], 
        ascending=[True, True, True, True])
    groupted_df_results = sorted_df_results[(sorted_df_results["rank_test_eer"] <= n_top)].groupby(['owner', 'run']).first()
    
    # Prepare data for plotting
    df_plot = df_results[df_results["rank_test_eer"] <= n_top].rename(
        columns={
                "param_model__n_neighbors": "n_neighbors",
                "param_model__contamination": "contamination",
                "mean_test_accuracy": "Mean Test Acc.",
                "mean_test_eer": "Mean Test EER",
        }
    )
    df_plot["Mean Test EER"] = df_plot["Mean Test EER"] * -1  # Because fewer is more

    median_n_neighbors = df_plot["n_neighbors"].median()
    median_contamination = df_plot["contamination"].median()

    
    fig=None
    if plot:
        fig=utils_plot_2d_hyperparam_selection("n_neighbors", median_n_neighbors, "contamination", median_contamination, df_plot,
                                               h1_min_max_scale_tuple=(-5, 50, "linear"), h2_min_max_scale_tuple=(-.1, 1.1, "linear"))
    

    return {"median_n_neighbors": median_n_neighbors, "median_contamination": median_contamination, "fig": fig}

In [2]:
def utils_plot_2d_hyperparam_selection(h1_name, h1_val, h2_name, h2_val, df_plot, 
                                       h1_min_max_scale_tuple=(None, None, "linear"), h2_min_max_scale_tuple=(None, None, "log")):
    
    h1_min, h1_max, h1_scale = h1_min_max_scale_tuple
    h2_min, h2_max, h2_scale = h2_min_max_scale_tuple
    
    
    if h2_scale=="log":
        h2_text_y = h2_val * np.power(h2_max / h2_min, 0.06)
        h2_text_2_y = h2_val * np.power(h2_max / h2_min, 0.02)
        
        h1_text_y = h2_val * np.power(h2_max / h2_min, 0.26)
        h1_text_2_y = h2_val * np.power(h2_max / h2_min, 0.22)
    
    elif h2_scale=="linear":
        h2_text_y = h2_val+ 0.07 * (h2_max - h2_min)
        h2_text_2_y = h2_val + 0.02 * (h2_max - h2_min)
        
        h1_text_y = h2_val+ 0.17 * (h2_max - h2_min)
        h1_text_2_y = h2_val + 0.12 * (h2_max - h2_min)
        
    else:
        raise Exception("Need to specify h2_scale")
    
    
    # Plot
    fig = plt.figure(figsize=(5.473 / 1.3, 2), dpi=180)
    g = sns.scatterplot(
        x=h1_name,
        y=h2_name,
        data=df_plot,
        size="Mean Test EER",
        sizes=(7, 60),
        hue="Mean Test EER",
        alpha=1,
        #        palette="Blues",
        linewidth=0,
    )

    # Format Legend labels
    leg = g.get_legend()
    new_handles = [h for h in leg.legendHandles]
    new_labels = []
    for i, handle in enumerate(leg.legendHandles):
        label = handle.get_label()
        print(f'{i}, {label}')
        if ord(label[0]) == 8722:
            label = '-' + label[1:]
            
        try:
            new_labels.append(f"{abs(float(label)):.3f}")

        except ValueError:
            new_labels.append("")

    # Plot mean values
    plt.plot(
        [h1_min-0.01, h1_max+0.01],
        [h2_val, h2_val],
        linestyle="dashed",
        linewidth=0.8,
        alpha=0.7,
        color="black",
    )
    plt.text(
        0.23,
        h2_text_y,
        f"median({h2_name})",
        fontsize=6,
        color="black",
        alpha=0.9,
    )
    plt.text(
        0.23,
        h2_text_2_y,
        f"{h2_val:.9f}",
        fontsize=5,
        color="black",
        alpha=0.9,
    )

    plt.plot(
        [h1_val, h1_val],
        [h2_min, h2_max],
        linestyle="dashed",
        linewidth=0.8,
        alpha=0.7,
        color="black",
    )
    plt.text(
        h1_val + 0.005, h1_text_y, f"median({h1_name})", fontsize=6, color="black", alpha=0.9
    )
    plt.text(
        h1_val + 0.005, h1_text_2_y, f"{h1_val:.9f}", fontsize=5, color="black", alpha=0.9
    )

    # Adjust axes & legend
    
    plt.xlim(h1_min, h1_max)
    plt.xscale(h1_scale)
    
    plt.ylim(h2_min, h2_max)
    plt.yscale(h2_scale)
    
#     print(new_handles)
    print(new_labels)
    plt.legend(
        new_handles,
        new_labels,
        bbox_to_anchor=(1.02, 1),
        loc=2,
        borderaxespad=0.0,
        title="Mean EER per Owner\n(Validation Results)",
        title_fontsize=5,
    )

    fig.tight_layout()
    
    return fig

In [63]:
# https://stackoverflow.com/questions/11159436/multiple-figures-in-a-single-window
def arrange_figures(figures, nrows = 1, ncols=1):
    """Plot a dictionary of figures.

    Parameters
    ----------
    figures : <title, figure> dictionary
    ncols : number of columns of subplots wanted in the display
    nrows : number of rows of subplots wanted in the figure
    """

    fig, axeslist = plt.subplots(ncols=ncols, nrows=nrows)
    for ind,title in enumerate(figures):
        axeslist.ravel()[ind].imshow(figures[title], cmap=plt.gray())
        axeslist.ravel()[ind].set_title(title)
        axeslist.ravel()[ind].set_axis_off()
        
    return fig

In [65]:
def utils_plot_acc_eer_dist(df_plot, y_col):
    n_subject = len(df_plot["Owner"].unique()) - 1
    mean_col = df_plot[y_col].mean()

    fig = plt.figure(figsize=(5.473, 2), dpi=180)
    ax = sns.boxplot(x="Owner", y=y_col, data=df_plot, **utils_boxplot_style)
    ax.set_ylim((0, 1))

    plt.plot(
        [-0.6, n_subject + 0.6],
        [mean_col, mean_col],
        linestyle="dashed",
        linewidth=1,
        color=MAGENTA,
        alpha=0.7,
    )
    plt.text(n_subject + 0.6, mean_col, f"mean", fontsize=6, color=MAGENTA)
    plt.text(
        n_subject + 0.6, mean_col - 0.04, f"{mean_col:.3f}", fontsize=4.5, color=MAGENTA
    )
    plt.xticks(rotation=45)
    fig.tight_layout()
    
    print(f"Overall mean: {mean_col:.4f}")
    return fig



# X_vals_owner_idx['X_train']
def utils_plot_acc_eer_dist_thesis(EER_distro_df, y_col, discription, save_file_name=None, boxplot_color="springgreen"):

    sns.set(err_distro_rc)
    n_subject = len(EER_distro_df['owner'].unique())+1
    mean_col = EER_distro_df[y_col].mean()
    fig = plt.figure()
    ax = sns.boxplot(x="owner", y=y_col, data=EER_distro_df, color=boxplot_color)#, **utils_boxplot_style)
    
    # Select which box you want to change    
    mybox = ax.patches[4]

    ax.set_ylim((0, 1))
    sns.swarmplot(x="owner", y=y_col, data=EER_distro_df, color=".25")
    

    plt.plot(
        [-0.5, n_subject -1.5],
        [mean_col, mean_col],
        linestyle="dashed",
        linewidth=2,
        color=MAGENTA,
        alpha=0.7,
    )
    plt.text(n_subject/2, mean_col + 0.01, f"mean", fontsize=err_distro_rc["ytick.labelsize"], color=MAGENTA)
    plt.text(
        n_subject/2, mean_col - 0.04, f"{mean_col:.3f}", fontsize=err_distro_rc["ytick.labelsize"], color=MAGENTA
    )
    plt.xticks(rotation=45)
    fig.tight_layout()
    plt.title(discription)
    
    ax.set_xlabel("owner id")
    ax.set_ylabel(f"Mean {y_col}")
    
    plt.savefig(f'{save_file_name}', bbox_inches='tight')
    print(f"Overall mean: {mean_col:.4f}")
    

In [66]:
# if TEST_MODE:
#     print("Performing Dummy RandomSearch...")
#     from sklearn import svm, datasets
#     from sklearn.model_selection import RandomizedSearchCV

#     iris = datasets.load_iris()
#     parameters = {"kernel": ("linear", "rbf"), "C": [1, 2, 3, 4, 5, 6, 7, 10]}
#     svc = svm.SVC(gamma="scale")
#     clf = RandomizedSearchCV(svc, parameters, cv=3, iid=False)
#     clf.fit(iris.data, iris.target)
#     print("Create report:")
#     df_temp = utils_cv_report(clf, "owner x", ["impo_1", "impo_2", "impo_3"])
#     display(df_temp)

In [1]:
def make_eer_per_window_size_col_df(df, key_column, window_size_lst):
    df = df.copy()

    # Calculate the mean and variance of EER for each unique Key
    agg_df = df.groupby(key_column)['Mean_EER'].agg(['mean', 'var']).reset_index()

    # Pivot the original DataFrame
    pivot_df = df.pivot_table(index=key_column, columns='window_size', values='Mean_EER').reset_index()

    # Concatenate the pivoted DataFrame with the mean and variance columns
    df = pd.concat([pivot_df, agg_df[['mean', 'var']]], axis=1)

    # Rename the columns
    columns = [('Window Size', col) if col in window_size_lst else ("", col) for col in df.columns]
    multiindex_columns = pd.MultiIndex.from_tuples(columns)

    df.columns = multiindex_columns
    
    sorted_df = df.sort_values(("", 'mean'), ascending=True).reset_index(drop=True)
    sorted_df.index.name="mean rank"
    
    return sorted_df

# Iterate through the dictionary and add the key as a new column 'Key' in each DataFrame
def make_raw_exp_df_results(EER_df_dict, key_column):
    
    dfs = []
    for key, df in EER_df_dict.items():
        df=df.copy()
        
        try:
            df[key_column] = key
        
        except Exception as e:
            print(e)
            print("the number of columns in key_column must match the number of elements of dict key!")
            print(f"len(key_column): {len(key_column)} != len(key): {len(key)}")
            print(f"type(key_column): {type(key_column)}, type(key): {type(key)}")
            return
        dfs.append(df)

    # Concatenate all the DataFrames
    return pd.concat(dfs, ignore_index=True)


In [ ]:
def return_and_save_final_result_df_as_json(final_exp_results_path, exp_path_name, window_size_lst):
    window_size_cols=[('Window Size', winsize) for winsize in window_size_lst]
    preprocessing_methods=["Naive", "Realworld-per_unknown_window"]
    smoothing_methods=["Butterworth", "EMA", "SMA", "Butter+EMA", "Butter+SMA"]
    concate_df_lst=[]
    
    
    test_file_name=f"{final_exp_results_path}/{exp_path_name}/None_Mean_EER_None_df_test_dict.txt"
    eer_per_window_size_col_df = pd.read_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json').drop(["('', 'mean')", "('', 'var')"], axis=1)#"('index', '')", 
    # print(eer_per_window_size_col_df)
    # Convert the string representation of tuples back to tuples
    idx_tuples = [eval(i) for i in eer_per_window_size_col_df.columns.tolist()]

    # Convert list of tuples back to a MultiIndex
    multiindex_columns = pd.MultiIndex.from_tuples(idx_tuples)

    eer_per_window_size_col_df.columns = multiindex_columns

    eer_per_window_size_col_df[("", 'type')] = 'None'
    
    concate_df_lst.append(eer_per_window_size_col_df)
            
            
    for sm in smoothing_methods:
        for pm in preprocessing_methods:
            test_file_name=f"{final_exp_results_path}/{exp_path_name}/{sm}_Mean_EER_{pm}_df_test_dict.txt"
            eer_per_window_size_col_df = pd.read_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json').drop(["('', 'mean')", "('', 'var')"], axis=1)#"('index', '')", 
            # print(eer_per_window_size_col_df)
            # Convert the string representation of tuples back to tuples
            idx_tuples = [eval(i) for i in eer_per_window_size_col_df.columns.tolist()]

            # Convert list of tuples back to a MultiIndex
            multiindex_columns = pd.MultiIndex.from_tuples(idx_tuples)

            eer_per_window_size_col_df.columns = multiindex_columns

            if pm =="Realworld-per_unknown_window":
                pm="Real"
            eer_per_window_size_col_df[("", 'type')] = f'{pm}'

            concate_df_lst.append(eer_per_window_size_col_df)

    df = pd.concat(concate_df_lst)
    # # add a new column 'Mean' which is the mean of each row across the columns '125', '1500', and '2000'
    # df['Mean'] = df[['125', '1500', '2000']].mean(axis=1)
    df=df[[
        (           '', 'cut_off_freq'),
        (           '',     'EMA_span'),
        (           '',  'SMA_winsize'),
        (           '', 'type'),
        ]+window_size_cols]

    df = df.fillna(DASH_MACRO_NUM).reset_index(drop=True)

    df[("", 'cut_off_freq')] = df[("", 'cut_off_freq')].astype(np.int64)
    df[("", 'EMA_span')] = df[("", 'EMA_span')].astype(np.int64)
    df[("", 'SMA_winsize')] = df[("", 'SMA_winsize')].astype(np.int64)

    df[("", "mean")] = df[window_size_cols].mean(axis=1)
    df[("", "variance")] = df[window_size_cols].var(axis=1)

    df[("", 'mean rank')] = df[("", "mean")].rank(method='min').astype(np.int64)
    cols = list(df.columns)
    df = df[[cols[-1]] + cols[:-1]]

    df.replace(DASH_MACRO_NUM, "-", inplace=True)

    df.to_json(f"{final_exp_results_path}/{exp_path_name}/{exp_path_name}-df.json")
    
    return df

def return_and_save_final_relative_result_df_as_json(df, base_case_index, final_exp_results_path, exp_path_name, window_size_lst):
    window_size_cols=[('Window Size', winsize) for winsize in window_size_lst]
    # columns you want to apply the operation to
    selected_columns = window_size_cols+[("", "mean"), ("", "variance")]

    # apply the operation to selected columns
    df_selected = df[selected_columns]
    # get the other columns
    df_others = df.drop(columns=selected_columns)

    # Convert the DataFrame to percentage improvement relative to base case
    df_relative = df_selected.loc[base_case_index].subtract(df_selected)

    # Convert to percentage improvement
    df_relative = df_relative.divide(df_selected.loc[base_case_index], axis=1) * 100

    # concatenate the dataframes along the column axis
    df_relative = pd.concat([df_others, df_relative], axis=1)
    
    df_relative.to_json(f"{final_exp_results_path}/{exp_path_name}/{exp_path_name}-relative_df.json")
    
    return df_relative



In [ ]:
def make_Gini_per_window_size_col_df(df, window_size_lst):
    df = df.copy()
    cols=df.columns


    if "cut_off_freq" in cols and "EMA_span" in cols:
        key_column=["cut_off_freq", "EMA_span"]
        
    elif "cut_off_freq" in cols and "SMA_winsize" in cols:
        key_column=["cut_off_freq", "SMA_winsize"]
        
    elif "EMA_span" in cols:
        key_column=["EMA_span"]
        
    elif "SMA_winsize" in cols: 
        key_column=["SMA_winsize"]
        
    elif "cut_off_freq" in cols:
        key_column=["cut_off_freq"]

    
    # Calculate the mean and variance of Gini_coef for each unique Key
    agg_df = df.groupby(key_column)['Gini_coef'].agg(['mean', 'var']).reset_index()

    # Pivot the original DataFrame
    pivot_df = df.pivot_table(index=key_column, columns='window_size', values='Gini_coef').reset_index()

    # Concatenate the pivoted DataFrame with the mean and variance columns
    df = pd.concat([pivot_df, agg_df[['mean', 'var']]], axis=1)

    # Rename the columns
    columns = [('Window Size', col) if col in window_size_lst else ("", col) for col in df.columns]
    multiindex_columns = pd.MultiIndex.from_tuples(columns)

    df.columns = multiindex_columns
    
    sorted_df = df.sort_values(("", 'mean'), ascending=True).reset_index(drop=True)
    sorted_df.index.name="mean rank"
    
    return sorted_df

def return_and_save_final_Gini_df_as_json(final_exp_results_path, exp_path_name, window_size_lst):
    window_size_cols=[('Window Size', winsize) for winsize in window_size_lst]
    preprocessing_methods=["Naive", "Realworld-per_unknown_window"]
    smoothing_methods=["Butterworth", "EMA", "SMA", "Butter+EMA", "Butter+SMA"]
    concate_df_lst=[]
    
    
    test_file_name=f"{final_exp_results_path}/{exp_path_name}/None_Mean_EER_None_df_test_dict.txt"
    raw_result_df = pd.read_json(f'{test_file_name[:-4]}_raw_df.json')
    gini_per_window_size_col_df=make_Gini_per_window_size_col_df(raw_result_df, window_size_lst)

    # print(eer_per_window_size_col_df)
#     # Convert the string representation of tuples back to tuples
#     idx_tuples = [eval(i) for i in gini_per_window_size_col_df.columns.tolist()]

#     # Convert list of tuples back to a MultiIndex
#     multiindex_columns = pd.MultiIndex.from_tuples(idx_tuples)

#     gini_per_window_size_col_df.columns = multiindex_columns

    gini_per_window_size_col_df[("", 'type')] = 'None'
    
    concate_df_lst.append(gini_per_window_size_col_df)
            
            
    for sm in smoothing_methods:
        for pm in preprocessing_methods:
            test_file_name=f"{final_exp_results_path}/{exp_path_name}/{sm}_Mean_EER_{pm}_df_test_dict.txt"
            
            raw_result_df = pd.read_json(f'{test_file_name[:-4]}_raw_df.json')
            gini_per_window_size_col_df=make_Gini_per_window_size_col_df(raw_result_df, window_size_lst)
            
            # print(eer_per_window_size_col_df)
            # Convert the string representation of tuples back to tuples
#             idx_tuples = [eval(i) for i in gini_per_window_size_col_df.columns.tolist()]

#             # Convert list of tuples back to a MultiIndex
#             multiindex_columns = pd.MultiIndex.from_tuples(idx_tuples)

#             gini_per_window_size_col_df.columns = multiindex_columns

            if pm =="Realworld-per_unknown_window":
                pm="Real"
            gini_per_window_size_col_df[("", 'type')] = f'{pm}'

            concate_df_lst.append(gini_per_window_size_col_df)

    df = pd.concat(concate_df_lst)
    # # add a new column 'Mean' which is the mean of each row across the columns '125', '1500', and '2000'
    # df['Mean'] = df[['125', '1500', '2000']].mean(axis=1)
    df=df[[
        (           '', 'cut_off_freq'),
        (           '',     'EMA_span'),
        (           '',  'SMA_winsize'),
        (           '', 'type'),
        ]+window_size_cols]

    df = df.fillna(DASH_MACRO_NUM).reset_index(drop=True)

    df[("", 'cut_off_freq')] = df[("", 'cut_off_freq')].astype(np.int64)
    df[("", 'EMA_span')] = df[("", 'EMA_span')].astype(np.int64)
    df[("", 'SMA_winsize')] = df[("", 'SMA_winsize')].astype(np.int64)

    df[("", "mean")] = df[window_size_cols].mean(axis=1)
    df[("", "variance")] = df[window_size_cols].var(axis=1)

    df[("", 'mean rank')] = df[("", "mean")].rank(method='min').astype(np.int64)
    cols = list(df.columns)
    df = df[[cols[-1]] + cols[:-1]]

    df.replace(DASH_MACRO_NUM, "-", inplace=True)

    df.to_json(f"{final_exp_results_path}/{exp_path_name}/{exp_path_name}-gini-df.json")
    
    return df

def return_and_save_final_relative_gini_result_df_as_json(df, base_case_index, final_exp_results_path, exp_path_name, window_size_lst):
    window_size_cols=[('Window Size', winsize) for winsize in window_size_lst]
    # columns you want to apply the operation to
    selected_columns = window_size_cols+[("", "mean"), ("", "variance")]

    # apply the operation to selected columns
    df_selected = df[selected_columns]
    # get the other columns
    df_others = df.drop(columns=selected_columns)

    # Convert the DataFrame to percentage improvement relative to base case
    df_relative = df_selected.loc[base_case_index].subtract(df_selected)

    # Convert to percentage improvement
    df_relative = df_relative.divide(df_selected.loc[base_case_index], axis=1) * 100

    # concatenate the dataframes along the column axis
    df_relative = pd.concat([df_others, df_relative], axis=1)
    
    df_relative.to_json(f"{final_exp_results_path}/{exp_path_name}/{exp_path_name}-gini-relative_df.json")
    
    return df_relative



In [14]:
print(20*'-' + "\x1b[32mClassification utility functions imported\x1b[0m" + 20*'-')

--------------------Classification utility functions imported--------------------


In [ ]:
["cut_off_freq"]
["cut_off_freq", "EMA_span"]
["EMA_span"]
["SMA_winsize"]
["cut_off_freq", "SMA_winsize"]